In [10]:
# #from efficientnet import EfficientNetB4
# # import efficientnet
# # #!pip install --upgrade efficientnet
# # dir()
# # import keras
# # import tensorflow as tf

# tf.keras.applications.EfficientNetB0#efficientnet
# # efficientnet.tfkeras.EfficientNetB1#tfkeras#model#keras#init_keras_custom_objects#.tfkeras#.EfficientNetB0
# len(tf.keras.applications.EfficientNetB4((224,224,3)).layers)
1e-2


0.01

In [150]:
!pip uninstall efficientnet

^C


In [151]:
!pip install -U segmentation-models

Requirement already up-to-date: segmentation-models in c:\users\roy\anaconda3\envs\tensorflow_2.x.2\lib\site-packages (1.0.1)


In [189]:
import segmentation_models as sm

In [217]:
model = sm.FPN('efficientnetb2', encoder_weights='imagenet')

In [218]:
model.summary()
model.save("fpnetub2")

Model: "model_27"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_71 (InputLayer)           [(None, None, None,  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, None, None, 3 864         input_71[0][0]                   
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, None, None, 3 128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, None, None, 3 0           stem_bn[0][0]                    
___________________________________________________________________________________________

block6c_se_reshape (Reshape)    (None, 1, 1, 1248)   0           block6c_se_squeeze[0][0]         
__________________________________________________________________________________________________
block6c_se_reduce (Conv2D)      (None, 1, 1, 52)     64948       block6c_se_reshape[0][0]         
__________________________________________________________________________________________________
block6c_se_expand (Conv2D)      (None, 1, 1, 1248)   66144       block6c_se_reduce[0][0]          
__________________________________________________________________________________________________
block6c_se_excite (Multiply)    (None, None, None, 1 0           block6c_activation[0][0]         
                                                                 block6c_se_expand[0][0]          
__________________________________________________________________________________________________
block6c_project_conv (Conv2D)   (None, None, None, 2 259584      block6c_se_excite[0][0]          
__________

INFO:tensorflow:Assets written to: fpnetub2\assets


INFO:tensorflow:Assets written to: fpnetub2\assets


In [220]:
#model=keras.models.load_model("effnetub3")
#model.summary()
tf.keras.utils.plot_model(model, show_shapes=True)


('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [126]:
# input_shape=(224,224,3)
# efb0=tf.keras.applications.EfficientNetB0
# efb0(weights='imagenet', include_top=False, input_shape=input_shape)
from tensorflow.keras.layers import Input, Conv2D, ReLU, LeakyReLU, BatchNormalization, MaxPooling2D,\
    Flatten, Dense, Reshape, Conv2DTranspose, Activation, Dropout, Add, concatenate

def UEfficientNet(input_shape=(None, None, 3),dropout_rate=0.1,EfficientNetB4=tf.keras.applications.EfficientNetB4):

    backbone = EfficientNetB4(weights='imagenet',
                            include_top=False,
                            input_shape=input_shape)
    input = backbone.input
    start_neurons = 16

    conv4 = backbone.layers[342].output
    
    
    
    conv4 = LeakyReLU(alpha=0.1)(conv4)
    pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(dropout_rate)(pool4)
    
     # Middle
    convm = Conv2D(start_neurons * 32, (3, 3), activation=None, padding="same")(pool4)
    convm = residual_block(convm,start_neurons * 32)
    convm = residual_block(convm,start_neurons * 32)
    convm = LeakyReLU(alpha=0.1)(convm)
    
    deconv4 = Conv2DTranspose(start_neurons * 16, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv4 = concatenate([deconv4, conv4])
    uconv4 = Dropout(dropout_rate)(uconv4)
    
    uconv4 = Conv2D(start_neurons * 16, (3, 3), activation=None, padding="same")(uconv4)
    uconv4 = residual_block(uconv4,start_neurons * 16)
    uconv4 = residual_block(uconv4,start_neurons * 16)
    uconv4 = LeakyReLU(alpha=0.1)(uconv4)
    
    deconv3 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(uconv4)
    conv3 = backbone.layers[154].output
    
    
    
    
    uconv3 = concatenate([deconv3, conv3])    
    uconv3 = Dropout(dropout_rate)(uconv3)
    
    uconv3 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(uconv3)
    uconv3 = residual_block(uconv3,start_neurons * 8)
    uconv3 = residual_block(uconv3,start_neurons * 8)
    uconv3 = LeakyReLU(alpha=0.1)(uconv3)

    deconv2 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv3)
    conv2 = backbone.layers[92].output
    
    
    
    
    uconv2 = concatenate([deconv2, conv2])
        
    uconv2 = Dropout(0.1)(uconv2)
    uconv2 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(uconv2)
    uconv2 = residual_block(uconv2,start_neurons * 4)
    uconv2 = residual_block(uconv2,start_neurons * 4)
    uconv2 = LeakyReLU(alpha=0.1)(uconv2)
    
    deconv1 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv2)
    conv1 = backbone.layers[30].output
    
    
    
    
    uconv1 = concatenate([deconv1, conv1])
    
    uconv1 = Dropout(0.1)(uconv1)
    uconv1 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv1)
    uconv1 = residual_block(uconv1,start_neurons * 2)
    uconv1 = residual_block(uconv1,start_neurons * 2)
    uconv1 = LeakyReLU(alpha=0.1)(uconv1)
    
    uconv0 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv1)   
    uconv0 = Dropout(0.1)(uconv0)
    uconv0 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv0)
    uconv0 = residual_block(uconv0,start_neurons * 1)
    uconv0 = residual_block(uconv0,start_neurons * 1)
    uconv0 = LeakyReLU(alpha=0.1)(uconv0)
    
    uconv0 = Dropout(dropout_rate/2)(uconv0)
    #output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv0)    
    output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv0)    
    
    model = Model(input, output_layer)
    model.name = 'u-xception'

    return model

In [83]:
def convolution_block(x, filters, size, strides=(1,1), padding='same', activation=True):
    x = Conv2D(filters, size, strides=strides, padding=padding)(x)
    x = BatchNormalization()(x)
    if activation == True:
        x = LeakyReLU(alpha=0.1)(x)
    return x

def residual_block(blockInput, num_filters=16):
    x = LeakyReLU(alpha=0.1)(blockInput)
    x = BatchNormalization()(x)
    blockInput = BatchNormalization()(blockInput)
    x = convolution_block(x, num_filters, (3,3) )
    x = convolution_block(x, num_filters, (3,3), activation=False)
    x = Add()([x, blockInput])
    return x

In [257]:
#del model 
# model=keras.models.load_model("effnetub0")
# model.summary()
input_shape=(600,600,3)
# EfficientNetB4=tf.keras.applications.EfficientNetB4
# backbone = EfficientNetB4(weights='imagenet',
#                             include_top=False,
#                             input_shape=input_shape)
UEfficientNet(input_shape,dropout_rate=0.25)

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concat axis. Got inputs shapes: [(None, 18, 18, 256), (None, 19, 19, 1632)]

In [149]:
backbone.input[0]
backbone.get_layer('block7b_expand_bn')

In [59]:
backbone.summary()

Model: "efficientnetb4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling_15 (Rescaling)        (None, 224, 224, 3)  0           input_16[0][0]                   
__________________________________________________________________________________________________
normalization_15 (Normalization (None, 224, 224, 3)  7           rescaling_15[0][0]               
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization_15[0][0]           
_____________________________________________________________________________________

In [290]:
input_shape=(224,224,3)
#EfficientNetB4=tf.keras.applications.EfficientNetB4
backbone = EfficientNetB4(weights='imagenet',
                            include_top=False,
                            input_shape=input_shape)
dropout_rate=0.3
input = backbone.input
start_neurons = 16


conv4 = backbone.layers[342].output
conv4 = LeakyReLU(alpha=0.1)(conv4)
pool4 = MaxPooling2D((2, 2))(conv4)
pool4 = Dropout(dropout_rate)(pool4)



# # Middle
# convm = Conv2D(start_neurons * 28, (3, 3), activation=None, padding="same")(pool4)
# convm = residual_block(convm,start_neurons * 28)
# convm = residual_block(convm,start_neurons * 28)
# convm = LeakyReLU(alpha=0.1)(convm)
    
# deconv4 = Conv2DTranspose(start_neurons * 14, (3, 3), strides=(2, 2), padding="same")(convm)
# deconv4.shape
# conv4.shape
# uconv4 = concatenate([deconv4, conv4])
# uconv4 = Dropout(dropout_rate)(uconv4)
    
# uconv4 = Conv2D(start_neurons * 14, (3, 3), activation=None, padding="same")(uconv4)
# uconv4 = residual_block(uconv4,start_neurons * 14)
# uconv4 = residual_block(uconv4,start_neurons * 14)
# uconv4 = LeakyReLU(alpha=0.1)(uconv4)
    
# deconv3 = Conv2DTranspose(start_neurons * 7, (3, 3), strides=(2, 2), padding="same")(uconv4)
# conv3 = backbone.layers[154].output
# uconv3 = concatenate([deconv3, conv3])    
# uconv3 = Dropout(dropout_rate)(uconv3)
    
# uconv3 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(uconv3)
# uconv3 = residual_block(uconv3,start_neurons * 8)
# uconv3 = residual_block(uconv3,start_neurons * 8)
# uconv3 = LeakyReLU(alpha=0.1)(uconv3)



In [291]:
len(backbone.layers)
pool4.shape
keras.Model(input, pool4).summary()

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_24 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling_9 (Rescaling)         (None, 224, 224, 3)  0           input_24[0][0]                   
__________________________________________________________________________________________________
normalization_9 (Normalization) (None, 224, 224, 3)  7           rescaling_9[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization_9[0][0]            
___________________________________________________________________________________________

In [314]:
[o.name for o in backbone.layers[330:370] ]
backbone.layers[10].compute_output_shape((None,112,112,48))
#backbone.layers[10].name
#[backbone.layers[i] for i in [342, 154,92, 30]]
# print(backbone.input.shape)
# print(conv4.shape)
#print(deconv4.shape)
# #concatenate([deconv4, conv4])

TensorShape([None, 48])

In [91]:
print(backbone.input.shape)
print(conv4.shape)
print(deconv4.shape)
concatenate([deconv4, conv4])

(None, 256, 256, 3)
(None, 8, 8, 1632)
(None, 8, 8, 256)


<KerasTensor: shape=(None, 8, 8, 1888) dtype=float32 (created by layer 'concatenate_14')>

In [244]:
encoder=tf.keras.applications.EfficientNetB7(input_shape=(224,224,3), include_top=False)
encoder.summary()

258080768/258076736 [==============================] - 31s 0us/step
Model: "efficientnetb7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 3)  0           input_7[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 224, 224, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization[0][0]              
_________________

block3f_project_bn (BatchNormal (None, 28, 28, 80)   320         block3f_project_conv[0][0]       
__________________________________________________________________________________________________
block3f_drop (Dropout)          (None, 28, 28, 80)   0           block3f_project_bn[0][0]         
__________________________________________________________________________________________________
block3f_add (Add)               (None, 28, 28, 80)   0           block3f_drop[0][0]               
                                                                 block3e_add[0][0]                
__________________________________________________________________________________________________
block3g_expand_conv (Conv2D)    (None, 28, 28, 480)  38400       block3f_add[0][0]                
__________________________________________________________________________________________________
block3g_expand_bn (BatchNormali (None, 28, 28, 480)  1920        block3g_expand_conv[0][0]        
__________

block6i_project_bn (BatchNormal (None, 7, 7, 384)    1536        block6i_project_conv[0][0]       
__________________________________________________________________________________________________
block6i_drop (Dropout)          (None, 7, 7, 384)    0           block6i_project_bn[0][0]         
__________________________________________________________________________________________________
block6i_add (Add)               (None, 7, 7, 384)    0           block6i_drop[0][0]               
                                                                 block6h_add[0][0]                
__________________________________________________________________________________________________
block6j_expand_conv (Conv2D)    (None, 7, 7, 2304)   884736      block6i_add[0][0]                
__________________________________________________________________________________________________
block6j_expand_bn (BatchNormali (None, 7, 7, 2304)   9216        block6j_expand_conv[0][0]        
__________

In [266]:
inputs = keras.Input(shape=(224,224,3))

    ### [First half of the network: downsampling inputs] ###

    # Entry block
x = layers.Conv2DTranspose(32, 3, strides=2, padding="same")(inputs)
print(x.shape)
# x = layers.BatchNormalization()(x)
# x = layers.Activation("relu")(x)

model = keras.Model(inputs, x)

# model.summary()


(None, 448, 448, 32)


In [247]:
len(encoder.layers)

813

In [152]:

def get_blocknr_of_skip_candidates(encoder, verbose=False):
    """
    Get block numbers of the blocks which will be used for concatenation in the Unet.
    :param encoder: the encoder
    :param verbose: if set to True, the shape information of all blocks will be printed in the console
    :return: a list of block numbers
    """
    shapes = []
    candidates = []
    mbblock_nr = 0
    while True:
        try:
            mbblock = encoder.get_layer('blocks_{}_output_batch_norm'.format(mbblock_nr)).output
            shape = int(mbblock.shape[1]), int(mbblock.shape[2])
            if shape not in shapes:
                shapes.append(shape)
                candidates.append(mbblock_nr)
            if verbose:
                print('blocks_{}_output_shape: {}'.format(mbblock_nr, shape))
            mbblock_nr += 1
        except ValueError:
            break
    return candidates


def DoubleConv(filters, kernel_size, initializer='glorot_uniform'):

    def layer(x):

        x = Conv2D(filters, kernel_size, padding='same', use_bias=False, kernel_initializer=initializer)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Conv2D(filters, kernel_size, padding='same', use_bias=False, kernel_initializer=initializer)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        return x

    return layer


def UpSampling2D_block(filters, kernel_size=(3, 3), upsample_rate=(2, 2), interpolation='bilinear',
                       initializer='glorot_uniform', skip=None):
    def layer(input_tensor):

        x = UpSampling2D(size=upsample_rate, interpolation=interpolation)(input_tensor)

        if skip is not None:
            x = Concatenate()([x, skip])

        x = DoubleConv(filters, kernel_size, initializer=initializer)(x)

        return x
    return layer


def Conv2DTranspose_block(filters, kernel_size=(3, 3), transpose_kernel_size=(2, 2), upsample_rate=(2, 2),
                          initializer='glorot_uniform', skip=None):
    def layer(input_tensor):

        x = Conv2DTranspose(filters, transpose_kernel_size, strides=upsample_rate, padding='same')(input_tensor)

        if skip is not None:
            x = Concatenate()([x, skip])

        x = DoubleConv(filters, kernel_size, initializer=initializer)(x)

        return x

    return layer


# noinspection PyTypeChecker
def _get_efficient_unet(encoder, out_channels=2, block_type='upsampling', concat_input=True):
    MBConvBlocks = []

    skip_candidates = get_blocknr_of_skip_candidates(encoder)

    for mbblock_nr in skip_candidates:
        mbblock = encoder.get_layer('blocks_{}_output_batch_norm'.format(mbblock_nr)).output
        MBConvBlocks.append(mbblock)

    # delete the last block since it won't be used in the process of concatenation
    MBConvBlocks.pop()

    input_ = encoder.input
    head = encoder.get_layer('head_swish').output
    blocks = [input_] + MBConvBlocks + [head]

    if block_type == 'upsampling':
        UpBlock = UpSampling2D_block
    else:
        UpBlock = Conv2DTranspose_block

    o = blocks.pop()
    o = UpBlock(512, initializer=conv_kernel_initializer, skip=blocks.pop())(o)
    o = UpBlock(256, initializer=conv_kernel_initializer, skip=blocks.pop())(o)
    o = UpBlock(128, initializer=conv_kernel_initializer, skip=blocks.pop())(o)
    o = UpBlock(64, initializer=conv_kernel_initializer, skip=blocks.pop())(o)
    if concat_input:
        o = UpBlock(32, initializer=conv_kernel_initializer, skip=blocks.pop())(o)
    else:
        o = UpBlock(32, initializer=conv_kernel_initializer, skip=None)(o)
    o = Conv2D(out_channels, (1, 1), padding='same', kernel_initializer=conv_kernel_initializer)(o)

    model = models.Model(encoder.input, o)

    return model


def get_efficient_unet_b0(input_shape, out_channels=2, pretrained=False, block_type='transpose', concat_input=True):
    """Get a Unet model with Efficient-B0 encoder
    :param input_shape: shape of input (cannot have None element)
    :param out_channels: the number of output channels
    :param pretrained: True for ImageNet pretrained weights
    :param block_type: "upsampling" to use UpSampling layer, otherwise use Conv2DTranspose layer
    :param concat_input: if True, input image will be concatenated with the last conv layer
    :return: an EfficientUnet_B0 model
    """
    encoder = get_efficientnet_b0_encoder(input_shape, pretrained=pretrained)
    model = _get_efficient_unet(encoder, out_channels, block_type=block_type, concat_input=concat_input)
    return model


def get_efficient_unet_b1(input_shape, out_channels=2, pretrained=False, block_type='transpose', concat_input=True):
    """Get a Unet model with Efficient-B1 encoder
    :param input_shape: shape of input (cannot have None element)
    :param out_channels: the number of output channels
    :param pretrained: True for ImageNet pretrained weights
    :param block_type: "upsampling" to use UpSampling layer, otherwise use Conv2DTranspose layer
    :param concat_input: if True, input image will be concatenated with the last conv layer
    :return: an EfficientUnet_B1 model
    """
    encoder = get_efficientnet_b1_encoder(input_shape, pretrained=pretrained)
    model = _get_efficient_unet(encoder, out_channels, block_type=block_type, concat_input=concat_input)
    return model


def get_efficient_unet_b2(input_shape, out_channels=2, pretrained=False, block_type='transpose', concat_input=True):
    """Get a Unet model with Efficient-B2 encoder
    :param input_shape: shape of input (cannot have None element)
    :param out_channels: the number of output channels
    :param pretrained: True for ImageNet pretrained weights
    :param block_type: "upsampling" to use UpSampling layer, otherwise use Conv2DTranspose layer
    :param concat_input: if True, input image will be concatenated with the last conv layer
    :return: an EfficientUnet_B2 model
    """
    encoder = get_efficientnet_b2_encoder(input_shape, pretrained=pretrained)
    model = _get_efficient_unet(encoder, out_channels, block_type=block_type, concat_input=concat_input)
    return model


def get_efficient_unet_b3(input_shape, out_channels=2, pretrained=False, block_type='transpose', concat_input=True):
    """Get a Unet model with Efficient-B3 encoder
    :param input_shape: shape of input (cannot have None element)
    :param out_channels: the number of output channels
    :param pretrained: True for ImageNet pretrained weights
    :param block_type: "upsampling" to use UpSampling layer, otherwise use Conv2DTranspose layer
    :param concat_input: if True, input image will be concatenated with the last conv layer
    :return: an EfficientUnet_B3 model
    """
    encoder = get_efficientnet_b3_encoder(input_shape, pretrained=pretrained)
    model = _get_efficient_unet(encoder, out_channels, block_type=block_type, concat_input=concat_input)
    return model


def get_efficient_unet_b4(input_shape, out_channels=2, pretrained=False, block_type='transpose', concat_input=True):
    """Get a Unet model with Efficient-B4 encoder
    :param input_shape: shape of input (cannot have None element)
    :param out_channels: the number of output channels
    :param pretrained: True for ImageNet pretrained weights
    :param block_type: "upsampling" to use UpSampling layer, otherwise use Conv2DTranspose layer
    :param concat_input: if True, input image will be concatenated with the last conv layer
    :return: an EfficientUnet_B4 model
    """
    encoder = get_efficientnet_b4_encoder(input_shape, pretrained=pretrained)
    model = _get_efficient_unet(encoder, out_channels, block_type=block_type, concat_input=concat_input)
    return model


def get_efficient_unet_b5(input_shape, out_channels=2, pretrained=False, block_type='transpose', concat_input=True):
    """Get a Unet model with Efficient-B5 encoder
    :param input_shape: shape of input (cannot have None element)
    :param out_channels: the number of output channels
    :param pretrained: True for ImageNet pretrained weights
    :param block_type: "upsampling" to use UpSampling layer, otherwise use Conv2DTranspose layer
    :param concat_input: if True, input image will be concatenated with the last conv layer
    :return: an EfficientUnet_B5 model
    """
    encoder = get_efficientnet_b5_encoder(input_shape, pretrained=pretrained)
    model = _get_efficient_unet(encoder, out_channels, block_type=block_type, concat_input=concat_input)
    return model


def get_efficient_unet_b6(input_shape, out_channels=2, pretrained=False, block_type='transpose', concat_input=True):
    """Get a Unet model with Efficient-B6 encoder
    :param input_shape: shape of input (cannot have None element)
    :param out_channels: the number of output channels
    :param pretrained: True for ImageNet pretrained weights
    :param block_type: "upsampling" to use UpSampling layer, otherwise use Conv2DTranspose layer
    :param concat_input: if True, input image will be concatenated with the last conv layer
    :return: an EfficientUnet_B6 model
    """
    encoder = get_efficientnet_b6_encoder(input_shape, pretrained=pretrained)
    model = _get_efficient_unet(encoder, out_channels, block_type=block_type, concat_input=concat_input)
    return model


def get_efficient_unet_b7(input_shape, out_channels=2, pretrained=False, block_type='transpose', concat_input=True):
    """Get a Unet model with Efficient-B7 encoder
    :param input_shape: shape of input (cannot have None element)
    :param out_channels: the number of output channels
    :param pretrained: True for ImageNet pretrained weights
    :param block_type: "upsampling" to use UpSampling layer, otherwise use Conv2DTranspose layer
    :param concat_input: if True, input image will be concatenated with the last conv layer
    :return: an EfficientUnet_B7 model
    """
    encoder = get_efficientnet_b7_encoder(input_shape, pretrained=pretrained)
    model = _get_efficient_unet(encoder, out_channels, block_type=block_type, concat_input=concat_input)
    return model

In [153]:
from keras.layers import *
from keras import models

In [154]:

def _efficientnet(input_shape, blocks_args_list, global_params):
    batch_norm_momentum = global_params.batch_norm_momentum
    batch_norm_epsilon = global_params.batch_norm_epsilon

    # Stem part
    model_input = layers.Input(shape=input_shape)
    x = layers.Conv2D(
        filters=round_filters(32, global_params),
        kernel_size=[3, 3],
        strides=[2, 2],
        kernel_initializer=conv_kernel_initializer,
        padding='same',
        use_bias=False,
        name='stem_conv2d'
    )(model_input)

    x = layers.BatchNormalization(
        momentum=batch_norm_momentum,
        epsilon=batch_norm_epsilon,
        name='stem_batch_norm'
    )(x)

    x = Swish(name='stem_swish')(x)

    # Blocks part
    idx = 0
    drop_rate = global_params.drop_connect_rate
    n_blocks = sum([blocks_args.num_repeat for blocks_args in blocks_args_list])
    drop_rate_dx = drop_rate / n_blocks

    for blocks_args in blocks_args_list:
        assert blocks_args.num_repeat > 0
        # Update block input and output filters based on depth multiplier.
        blocks_args = blocks_args._replace(
            input_filters=round_filters(blocks_args.input_filters, global_params),
            output_filters=round_filters(blocks_args.output_filters, global_params),
            num_repeat=round_repeats(blocks_args.num_repeat, global_params)
        )

        # The first block needs to take care of stride and filter size increase.
        x = MBConvBlock(blocks_args, global_params, idx, drop_connect_rate=drop_rate_dx * idx)(x)
        idx += 1

        if blocks_args.num_repeat > 1:
            blocks_args = blocks_args._replace(input_filters=blocks_args.output_filters, strides=[1, 1])

        for _ in range(blocks_args.num_repeat - 1):
            x = MBConvBlock(blocks_args, global_params, idx, drop_connect_rate=drop_rate_dx * idx)(x)
            idx += 1

    # Head part
    x = layers.Conv2D(
        filters=round_filters(1280, global_params),
        kernel_size=[1, 1],
        strides=[1, 1],
        kernel_initializer=conv_kernel_initializer,
        padding='same',
        use_bias=False,
        name='head_conv2d'
    )(x)

    x = layers.BatchNormalization(
        momentum=batch_norm_momentum,
        epsilon=batch_norm_epsilon,
        name='head_batch_norm'
    )(x)

    x = Swish(name='head_swish')(x)

    x = layers.GlobalAveragePooling2D(name='global_average_pooling2d')(x)

    if global_params.dropout_rate > 0:
        x = layers.Dropout(global_params.dropout_rate)(x)

    x = layers.Dense(
        global_params.num_classes,
        kernel_initializer=dense_kernel_initializer,
        activation='softmax',
        name='head_dense'
    )(x)

    model = models.Model(model_input, x)

    return model


def get_model_by_name(model_name, input_shape, classes=1000, pretrained=False):
    """Get an EfficientNet model by its name.
    """
    blocks_args, global_params = get_efficientnet_params(model_name, override_params={'num_classes': classes})
    model = _efficientnet(input_shape, blocks_args, global_params)

    try:
        if pretrained:
            weights = IMAGENET_WEIGHTS[model_name]
            weights_path = get_file(
                weights['name'],
                weights['url'],
                cache_subdir='models',
                md5_hash=weights['md5'],
            )
            model.load_weights(weights_path)
    except KeyError as e:
        print("NOTE: Currently model {} doesn't have pretrained weights, therefore a model with randomly initialized"
              " weights is returned.".format(e))

    return model


def _get_efficientnet_encoder(model_name, input_shape, pretrained=False):
    model = get_model_by_name(model_name, input_shape, pretrained=pretrained)
    encoder = models.Model(model.input, model.get_layer('global_average_pooling2d').output)
    encoder.layers.pop()  # remove GAP layer
    return encoder


def get_efficientnet_b0_encoder(input_shape, pretrained=False):
    return _get_efficientnet_encoder('efficientnet-b0', input_shape, pretrained=pretrained)


def get_efficientnet_b1_encoder(input_shape, pretrained=False):
    return _get_efficientnet_encoder('efficientnet-b1', input_shape, pretrained=pretrained)


def get_efficientnet_b2_encoder(input_shape, pretrained=False):
    return _get_efficientnet_encoder('efficientnet-b2', input_shape, pretrained=pretrained)


def get_efficientnet_b3_encoder(input_shape, pretrained=False):
    return _get_efficientnet_encoder('efficientnet-b3', input_shape, pretrained=pretrained)


def get_efficientnet_b4_encoder(input_shape, pretrained=False):
    return _get_efficientnet_encoder('efficientnet-b4', input_shape, pretrained=pretrained)


def get_efficientnet_b5_encoder(input_shape, pretrained=False):
    return _get_efficientnet_encoder('efficientnet-b5', input_shape, pretrained=pretrained)


def get_efficientnet_b6_encoder(input_shape, pretrained=False):
    return _get_efficientnet_encoder('efficientnet-b6', input_shape, pretrained=pretrained)


def get_efficientnet_b7_encoder(input_shape, pretrained=False):
    return _get_efficientnet_encoder('efficientnet-b7', input_shape, pretrained=pretrained)

In [223]:
#from keras.utils import get_file
get_efficientnet_b7_encoder((224,224,3))

KeyboardInterrupt: 

In [156]:
import re
from collections import namedtuple
from keras import layers
import keras.backend as K
import tensorflow as tf
import math
import numpy as np

GlobalParams = namedtuple('GlobalParams', ['batch_norm_momentum', 'batch_norm_epsilon', 'dropout_rate', 'num_classes',
                                           'width_coefficient', 'depth_coefficient', 'depth_divisor', 'min_depth',
                                           'drop_connect_rate'])
GlobalParams.__new__.__defaults__ = (None,) * len(GlobalParams._fields)

BlockArgs = namedtuple('BlockArgs', ['kernel_size', 'num_repeat', 'input_filters', 'output_filters', 'expand_ratio',
                                     'id_skip', 'strides', 'se_ratio'])
BlockArgs.__new__.__defaults__ = (None,) * len(BlockArgs._fields)

IMAGENET_WEIGHTS = {

    'efficientnet-b0': {
        'name': 'efficientnet-b0_imagenet_1000.h5',
        'url': 'https://github.com/qubvel/efficientnet/releases/download/v0.0.1/efficientnet-b0_imagenet_1000.h5',
        'md5': 'bca04d16b1b8a7c607b1152fe9261af7',
    },

    'efficientnet-b1': {
        'name': 'efficientnet-b1_imagenet_1000.h5',
        'url': 'https://github.com/qubvel/efficientnet/releases/download/v0.0.1/efficientnet-b1_imagenet_1000.h5',
        'md5': 'bd4a2b82f6f6bada74fc754553c464fc',
    },

    'efficientnet-b2': {
        'name': 'efficientnet-b2_imagenet_1000.h5',
        'url': 'https://github.com/qubvel/efficientnet/releases/download/v0.0.1/efficientnet-b2_imagenet_1000.h5',
        'md5': '45b28b26f15958bac270ab527a376999',
    },

    'efficientnet-b3': {
        'name': 'efficientnet-b3_imagenet_1000.h5',
        'url': 'https://github.com/qubvel/efficientnet/releases/download/v0.0.1/efficientnet-b3_imagenet_1000.h5',
        'md5': 'decd2c8a23971734f9d3f6b4053bf424',
    },

    'efficientnet-b4': {
        'name': 'efficientnet-b4_imagenet_1000.h5',
        'url': 'https://github.com/qubvel/efficientnet/releases/download/v0.0.1/efficientnet-b4_imagenet_1000.h5',
        'md5': '01df77157a86609530aeb4f1f9527949',
    },

    'efficientnet-b5': {
        'name': 'efficientnet-b5_imagenet_1000.h5',
        'url': 'https://github.com/qubvel/efficientnet/releases/download/v0.0.1/efficientnet-b5_imagenet_1000.h5',
        'md5': 'c31311a1a38b5111e14457145fccdf32',
    }

}


def round_filters(filters, global_params):
    """Round number of filters."""
    multiplier = global_params.width_coefficient
    divisor = global_params.depth_divisor
    min_depth = global_params.min_depth
    if not multiplier:
        return filters

    filters *= multiplier
    min_depth = min_depth or divisor
    new_filters = max(min_depth, int(filters + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_filters < 0.9 * filters:
        new_filters += divisor
    return int(new_filters)


def round_repeats(repeats, global_params):
    """Round number of repeats."""
    multiplier = global_params.depth_coefficient
    if not multiplier:
        return repeats
    return int(math.ceil(multiplier * repeats))


def get_efficientnet_params(model_name, override_params=None):
    """Get efficientnet params based on model name."""
    params_dict = {
        # (width_coefficient, depth_coefficient, resolution, dropout_rate)
        # Note: the resolution here is just for reference, its values won't be used.
        'efficientnet-b0': (1.0, 1.0, 224, 0.2),
        'efficientnet-b1': (1.0, 1.1, 240, 0.2),
        'efficientnet-b2': (1.1, 1.2, 260, 0.3),
        'efficientnet-b3': (1.2, 1.4, 300, 0.3),
        'efficientnet-b4': (1.4, 1.8, 380, 0.4),
        'efficientnet-b5': (1.6, 2.2, 456, 0.4),
        'efficientnet-b6': (1.8, 2.6, 528, 0.5),
        'efficientnet-b7': (2.0, 3.1, 600, 0.5),
    }
    if model_name not in params_dict.keys():
        raise KeyError('There is no model named {}.'.format(model_name))

    width_coefficient, depth_coefficient, _, dropout_rate = params_dict[model_name]

    blocks_args = [
        'r1_k3_s11_e1_i32_o16_se0.25', 'r2_k3_s22_e6_i16_o24_se0.25',
        'r2_k5_s22_e6_i24_o40_se0.25', 'r3_k3_s22_e6_i40_o80_se0.25',
        'r3_k5_s11_e6_i80_o112_se0.25', 'r4_k5_s22_e6_i112_o192_se0.25',
        'r1_k3_s11_e6_i192_o320_se0.25',
    ]
    global_params = GlobalParams(
        batch_norm_momentum=0.99,
        batch_norm_epsilon=1e-3,
        dropout_rate=dropout_rate,
        drop_connect_rate=0.2,
        num_classes=1000,
        width_coefficient=width_coefficient,
        depth_coefficient=depth_coefficient,
        depth_divisor=8,
        min_depth=None)

    if override_params:
        global_params = global_params._replace(**override_params)

    decoder = BlockDecoder()
    return decoder.decode(blocks_args), global_params


class BlockDecoder(object):
    """Block Decoder for readability."""

    @staticmethod
    def _decode_block_string(block_string):
        """Gets a block through a string notation of arguments."""
        assert isinstance(block_string, str)
        ops = block_string.split('_')
        options = {}
        for op in ops:
            splits = re.split(r'(\d.*)', op)
            if len(splits) >= 2:
                key, value = splits[:2]
                options[key] = value

        if 's' not in options or len(options['s']) != 2:
            raise ValueError('Strides options should be a pair of integers.')

        return BlockArgs(
            kernel_size=int(options['k']),
            num_repeat=int(options['r']),
            input_filters=int(options['i']),
            output_filters=int(options['o']),
            expand_ratio=int(options['e']),
            id_skip=('noskip' not in block_string),
            se_ratio=float(options['se']) if 'se' in options else None,
            strides=[int(options['s'][0]), int(options['s'][1])]
        )

    @staticmethod
    def _encode_block_string(block):
        """Encodes a block to a string."""
        args = [
            'r%d' % block.num_repeat,
            'k%d' % block.kernel_size,
            's%d%d' % (block.strides[0], block.strides[1]),
            'e%s' % block.expand_ratio,
            'i%d' % block.input_filters,
            'o%d' % block.output_filters
        ]
        if 0 < block.se_ratio <= 1:
            args.append('se%s' % block.se_ratio)
        if block.id_skip is False:
            args.append('noskip')
        return '_'.join(args)

    def decode(self, string_list):
        """Decodes a list of string notations to specify blocks inside the network.
        Args:
          string_list: a list of strings, each string is a notation of block.
        Returns:
          A list of namedtuples to represent blocks arguments.
        """
        assert isinstance(string_list, list)
        blocks_args = []
        for block_string in string_list:
            blocks_args.append(self._decode_block_string(block_string))
        return blocks_args

    def encode(self, blocks_args):
        """Encodes a list of Blocks to a list of strings.
        Args:
          blocks_args: A list of namedtuples to represent blocks arguments.
        Returns:
          a list of strings, each string is a notation of block.
        """
        block_strings = []
        for block in blocks_args:
            block_strings.append(self._encode_block_string(block))
        return block_strings


class Swish(layers.Layer):
    def __init__(self, name=None, **kwargs):
        super().__init__(name=name, **kwargs)

    def call(self, inputs, **kwargs):
        return tf.nn.swish(inputs)

    def get_config(self):
        config = super().get_config()
        config['name'] = self.name
        return config


def SEBlock(block_args, **kwargs):
    num_reduced_filters = max(
        1, int(block_args.input_filters * block_args.se_ratio))
    filters = block_args.input_filters * block_args.expand_ratio

    spatial_dims = [1, 2]

    try:
        block_name = kwargs['block_name']
    except KeyError:
        block_name = ''

    def block(inputs):
        x = inputs
        x = layers.Lambda(lambda a: K.mean(a, axis=spatial_dims, keepdims=True))(x)
        x = layers.Conv2D(
            num_reduced_filters,
            kernel_size=[1, 1],
            strides=[1, 1],
            kernel_initializer=conv_kernel_initializer,
            padding='same',
            name=block_name + 'se_reduce_conv2d',
            use_bias=True
        )(x)

        x = Swish(name=block_name + 'se_swish')(x)

        x = layers.Conv2D(
            filters,
            kernel_size=[1, 1],
            strides=[1, 1],
            kernel_initializer=conv_kernel_initializer,
            padding='same',
            name=block_name + 'se_expand_conv2d',
            use_bias=True
        )(x)

        x = layers.Activation('sigmoid')(x)
        out = layers.Multiply()([x, inputs])
        return out

    return block


class DropConnect(layers.Layer):

    def __init__(self, drop_connect_rate, **kwargs):
        super().__init__(**kwargs)
        self.drop_connect_rate = drop_connect_rate

    def call(self, inputs, **kwargs):
        def drop_connect():
            keep_prob = 1.0 - self.drop_connect_rate

            # Compute drop_connect tensor
            batch_size = tf.shape(inputs)[0]
            random_tensor = keep_prob
            random_tensor += tf.random.uniform([batch_size, 1, 1, 1], dtype=inputs.dtype)
            binary_tensor = tf.floor(random_tensor)
            output = tf.math.divide(inputs, keep_prob) * binary_tensor
            return output

        return K.in_train_phase(drop_connect(), inputs, training=None)

    def get_config(self):
        config = super().get_config()
        config['drop_connect_rate'] = self.drop_connect_rate
        return config


def conv_kernel_initializer(shape, dtype=K.floatx()):
    """Initialization for convolutional kernels.
    The main difference with tf.variance_scaling_initializer is that
    tf.variance_scaling_initializer uses a truncated normal with an uncorrected
    standard deviation, whereas here we use a normal distribution. Similarly,
    tf.contrib.layers.variance_scaling_initializer uses a truncated normal with
    a corrected standard deviation.
    Args:
        shape: shape of variable
        dtype: dtype of variable
    Returns:
        an initialization for the variable
    """
    kernel_height, kernel_width, _, out_filters = shape
    fan_out = int(kernel_height * kernel_width * out_filters)
    return tf.random.normal(
        shape, mean=0.0, stddev=np.sqrt(2.0 / fan_out), dtype=dtype)


def dense_kernel_initializer(shape, dtype=K.floatx()):
    init_range = 1.0 / np.sqrt(shape[1])
    return tf.random.uniform(shape, -init_range, init_range, dtype=dtype)


def MBConvBlock(block_args, global_params, idx, drop_connect_rate=None):
    filters = block_args.input_filters * block_args.expand_ratio
    batch_norm_momentum = global_params.batch_norm_momentum
    batch_norm_epsilon = global_params.batch_norm_epsilon
    has_se = (block_args.se_ratio is not None) and (0 < block_args.se_ratio <= 1)

    block_name = 'blocks_' + str(idx) + '_'

    def block(inputs):
        x = inputs

        # Expansion phase
        if block_args.expand_ratio != 1:
            expand_conv = layers.Conv2D(filters,
                                        kernel_size=[1, 1],
                                        strides=[1, 1],
                                        kernel_initializer=conv_kernel_initializer,
                                        padding='same',
                                        use_bias=False,
                                        name=block_name + 'expansion_conv2d'
                                        )(x)
            bn0 = layers.BatchNormalization(momentum=batch_norm_momentum,
                                            epsilon=batch_norm_epsilon,
                                            name=block_name + 'expansion_batch_norm')(expand_conv)

            x = Swish(name=block_name + 'expansion_swish')(bn0)

        # Depth-wise convolution phase
        kernel_size = block_args.kernel_size
        depthwise_conv = layers.DepthwiseConv2D(
            [kernel_size, kernel_size],
            strides=block_args.strides,
            depthwise_initializer=conv_kernel_initializer,
            padding='same',
            use_bias=False,
            name=block_name + 'depthwise_conv2d'
        )(x)
        bn1 = layers.BatchNormalization(momentum=batch_norm_momentum,
                                        epsilon=batch_norm_epsilon,
                                        name=block_name + 'depthwise_batch_norm'
                                        )(depthwise_conv)
        x = Swish(name=block_name + 'depthwise_swish')(bn1)

        if has_se:
            x = SEBlock(block_args, block_name=block_name)(x)

        # Output phase
        project_conv = layers.Conv2D(
            block_args.output_filters,
            kernel_size=[1, 1],
            strides=[1, 1],
            kernel_initializer=conv_kernel_initializer,
            padding='same',
            name=block_name + 'output_conv2d',
            use_bias=False)(x)
        x = layers.BatchNormalization(momentum=batch_norm_momentum,
                                      epsilon=batch_norm_epsilon,
                                      name=block_name + 'output_batch_norm'
                                      )(project_conv)
        if block_args.id_skip:
            if all(
                    s == 1 for s in block_args.strides
            ) and block_args.input_filters == block_args.output_filters:
                # only apply drop_connect if skip presents.
                if drop_connect_rate:
                    x = DropConnect(drop_connect_rate)(x)
                x = layers.add([x, inputs])

        return x

    return block


def freeze_efficientunet_first_n_blocks(model, n):
    mbblock_nr = 0
    while True:
        try:
            model.get_layer('blocks_{}_output_batch_norm'.format(mbblock_nr))
            mbblock_nr += 1
        except ValueError:
            break

    all_block_names = ['blocks_{}_output_batch_norm'.format(i) for i in range(mbblock_nr)]
    all_block_index = []
    for idx, layer in enumerate(model.layers):
        if layer.name == all_block_names[0]:
            all_block_index.append(idx)
            all_block_names.pop(0)
            if len(all_block_names) == 0:
                break
    n_blocks = len(all_block_index)

    if n <= 0:
        print('n is less than or equal to 0, therefore no layer will be frozen.')
        return
    if n > n_blocks:
        raise ValueError("There are {} blocks in total, n cannot be greater than {}.".format(n_blocks, n_blocks))

    idx_of_last_block_to_be_frozen = all_block_index[n - 1]
    for layer in model.layers[:idx_of_last_block_to_be_frozen + 1]:
        layer.trainable = False


def unfreeze_efficientunet(model):
    for layer in model.layers:
        layer.trainable = True

In [171]:
 model=get_efficient_unet_b0((224, 224, 3), pretrained=True, block_type='transpose', concat_input=True,out_channels=1)

In [1]:
%pwd
def save_load_pickle(pickfile, obj=None, save=True):
  import pickle
  if save:

    with open(pickfile, 'wb') as f:
        pickle.dump(obj, f) #, protocol=pickle.HIGHEST_PROTOCOL

  else:
    with open(pickfile, 'rb') as f:
        return pickle.load(f)

In [15]:
log=save_load_pickle("log.pickle",None,False)
log

{0: {'acc': [[<tf.Tensor 'Const_3016:0' shape=() dtype=float32>,
    <tf.Tensor 'Const_3017:0' shape=() dtype=float32>,
    <tf.Tensor 'Const_3018:0' shape=() dtype=float32>,
    <tf.Tensor 'Const_3019:0' shape=() dtype=float32>,
    <tf.Tensor 'Const_3020:0' shape=() dtype=float32>,
    <tf.Tensor 'Const_3021:0' shape=() dtype=float32>,
    <tf.Tensor 'Const_3022:0' shape=() dtype=float32>,
    <tf.Tensor 'Const_3023:0' shape=() dtype=float32>,
    <tf.Tensor 'Const_3024:0' shape=() dtype=float32>,
    <tf.Tensor 'Const_3025:0' shape=() dtype=float32>,
    <tf.Tensor 'Const_3026:0' shape=() dtype=float32>,
    <tf.Tensor 'Const_3027:0' shape=() dtype=float32>,
    <tf.Tensor 'Const_3028:0' shape=() dtype=float32>,
    <tf.Tensor 'Const_3029:0' shape=() dtype=float32>,
    <tf.Tensor 'Const_3030:0' shape=() dtype=float32>,
    <tf.Tensor 'Const_3031:0' shape=() dtype=float32>,
    <tf.Tensor 'Const_3032:0' shape=() dtype=float32>,
    <tf.Tensor 'Const_3033:0' shape=() dtype=float32>,


In [30]:
#model.summary()
import tensorflow as tf
# for i in log[0]['acc'][0]:
#     print(i)


AttributeError: type object 'h5py.h5a.AttrID' has no attribute '__reduce_cython__'

In [24]:
model.save("effunet_b0")
#del model
#np.save(model,"effunet_b0.h5.npz")

NameError: name 'model' is not defined

In [183]:
# del model

#type(model)
model=keras.models.load_model("effunet_b0")

In [221]:
model.summary()

Model: "model_27"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_71 (InputLayer)           [(None, None, None,  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, None, None, 3 864         input_71[0][0]                   
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, None, None, 3 128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, None, None, 3 0           stem_bn[0][0]                    
___________________________________________________________________________________________

__________________________________________________________________________________________________
block4c_expand_bn (BatchNormali (None, None, None, 5 2112        block4c_expand_conv[0][0]        
__________________________________________________________________________________________________
block4c_expand_activation (Acti (None, None, None, 5 0           block4c_expand_bn[0][0]          
__________________________________________________________________________________________________
block4c_dwconv (DepthwiseConv2D (None, None, None, 5 4752        block4c_expand_activation[0][0]  
__________________________________________________________________________________________________
block4c_bn (BatchNormalization) (None, None, None, 5 2112        block4c_dwconv[0][0]             
__________________________________________________________________________________________________
block4c_activation (Activation) (None, None, None, 5 0           block4c_bn[0][0]                 
__________

__________________________________________________________________________________________________
segm_stage4b_conv (Conv2D)      (None, None, None, 1 147456      segm_stage4a_relu[0][0]          
__________________________________________________________________________________________________
segm_stage5b_conv (Conv2D)      (None, None, None, 1 147456      segm_stage5a_relu[0][0]          
__________________________________________________________________________________________________
segm_stage2b_conv (Conv2D)      (None, None, None, 1 147456      segm_stage2a_relu[0][0]          
__________________________________________________________________________________________________
segm_stage3b_bn (BatchNormaliza (None, None, None, 1 512         segm_stage3b_conv[0][0]          
__________________________________________________________________________________________________
segm_stage4b_bn (BatchNormaliza (None, None, None, 1 512         segm_stage4b_conv[0][0]          
__________

In [233]:
from tensorflow.keras import layers


def get_model(img_size, num_classes):
    inputs = keras.Input(shape=img_size + (3,))

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [32, 64, 128, 256]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    ### [Second half of the network: upsampling inputs] ###

    for filters in [256, 128, 64, 32, 16]:
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.UpSampling2D(2)(x)

        # Project residual
        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding="same")(residual)
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    act="sigmoid"#"softmax"
    outputs = layers.Conv2D(num_classes, 3, activation=act, padding="same")(x)

    # Define the model
    model = keras.Model(inputs, outputs)
    return model
#Conv2D(n_label, (1, 1), padding='same', data_format=data_format)

# Free up RAM in case the model definition cells were run multiple times
keras.backend.clear_session()

# Build model
img_size=(224,224)
num_classes=1
model = get_model(img_size, num_classes)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 112, 112, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 112, 112, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [236]:
inputs = keras.Input(shape=img_size + (3,))

    ### [First half of the network: downsampling inputs] ###

    # Entry block
x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)

previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
for filters in [32,64, 128, 256]:
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(filters, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(filters, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
    residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
         previous_block_activation)
    x = layers.add([x, residual])  # Add back residual
    previous_block_activation = x  # Set aside next residual

    ### [Second half of the network: upsampling inputs] ###

for filters in [256, 128, 64, 32]:
    x = layers.Activation("relu")(x)
    x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)

    x = layers.Activation("relu")(x)
    x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)

    x = layers.UpSampling2D(2)(x)

        # Project residual
    residual = layers.UpSampling2D(2)(previous_block_activation)
    residual = layers.Conv2D(filters, 1, padding="same")(residual)
    x = layers.add([x, residual])  # Add back residual
    previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
act="sigmoid"#"softmax"
outputs = layers.Conv2D(num_classes, 3, activation=act, padding="same")(x)

    # Define the model
model = keras.Model(inputs, outputs)

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_31 (Conv2D)              (None, 112, 112, 32) 896         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 112, 112, 32) 128         conv2d_31[0][0]                  
__________________________________________________________________________________________________
activation_53 (Activation)      (None, 112, 112, 32) 0           batch_normalization_53[0][0]     
____________________________________________________________________________________________

In [315]:
[o.name for o in backbone.layers[330:370] ]
backbone.layers[10].compute_output_shape((None,112,112,48))

TensorShape([None, 48])

## https://www.kaggle.com/meaninglesslives/nested-unet-with-efficientnet-encoder

In [ ]:
for i in [a,b,c,d]:
    deconv4 = Conv2DTranspose(start_neurons * 16, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv4 = concatenate([deconv4, backbone.layers[i].output])
    
    
    uconv4 = Dropout(dropout_rate)(uconv4)
    uconv4 = Conv2D(start_neurons * 16, (3, 3), activation=None, padding="same")(uconv4)
    uconv4 = residual_block(uconv4,start_neurons * 16)
    uconv4 = residual_block(uconv4,start_neurons * 16)
    uconv4 = LeakyReLU(alpha=0.1)(uconv4)


In [ ]:
# input_shape=(224,224,3)
# efb0=tf.keras.applications.EfficientNetB0
# efb0(weights='imagenet', include_top=False, input_shape=input_shape)
from tensorflow.keras.layers import Input, Conv2D, ReLU, LeakyReLU, BatchNormalization, MaxPooling2D,\
    Flatten, Dense, Reshape, Conv2DTranspose, Activation, Dropout, Add, concatenate

def UEfficientNet(input_shape=(None, None, 3),dropout_rate=0.1,EfficientNetB0=tf.keras.applications.EfficientNetB0):

    encoder = EfficientNetB4(weights='imagenet',
                            include_top=False,
                            input_shape=input_shape)
    input = backbone.input
    start_neurons = 16

    conv4 = backbone.layers[342].output
    
    
    
    conv4 = LeakyReLU(alpha=0.1)(conv4)
    pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(dropout_rate)(pool4)
    
    # Middle

        
        
    convm = Conv2D(start_neurons * 32, (3, 3), activation=None, padding="same")(pool4)
    convm = residual_block(convm,start_neurons * 32)
    convm = residual_block(convm,start_neurons * 32)
    convm = LeakyReLU(alpha=0.1)(convm)
    
    
    
    
    
    deconv4 = Conv2DTranspose(start_neurons * 16, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv4 = concatenate([deconv4, conv4])
    
    
    uconv4 = Dropout(dropout_rate)(uconv4)
    uconv4 = Conv2D(start_neurons * 16, (3, 3), activation=None, padding="same")(uconv4)
    uconv4 = residual_block(uconv4,start_neurons * 16)
    uconv4 = residual_block(uconv4,start_neurons * 16)
    uconv4 = LeakyReLU(alpha=0.1)(uconv4)
    
    
    
    deconv3 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(uconv4)
    
    conv3 = backbone.layers[154].output
    
    uconv3 = concatenate([deconv3, conv3])    
    uconv3 = Dropout(dropout_rate)(uconv3)
    uconv3 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(uconv3)
    uconv3 = residual_block(uconv3,start_neurons * 8)
    uconv3 = residual_block(uconv3,start_neurons * 8)
    uconv3 = LeakyReLU(alpha=0.1)(uconv3)

    
    
    
    
    
    
    deconv2 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv3)
    
    
    conv2 = backbone.layers[92].output
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = Dropout(0.1)(uconv2)
    uconv2 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(uconv2)
    uconv2 = residual_block(uconv2,start_neurons * 4)
    uconv2 = residual_block(uconv2,start_neurons * 4)
    uconv2 = LeakyReLU(alpha=0.1)(uconv2)
    
    
    
    
    
    deconv1 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv2)
    
    
    
    
    
    
    
    conv1 = backbone.layers[30].output
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = Dropout(0.1)(uconv1)
    uconv1 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv1)
    uconv1 = residual_block(uconv1,start_neurons * 2)
    uconv1 = residual_block(uconv1,start_neurons * 2)
    uconv1 = LeakyReLU(alpha=0.1)(uconv1)
   










    uconv0 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv1)   
    uconv0 = Dropout(0.1)(uconv0)
    uconv0 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv0)
    uconv0 = residual_block(uconv0,start_neurons * 1)
    uconv0 = residual_block(uconv0,start_neurons * 1)
    uconv0 = LeakyReLU(alpha=0.1)(uconv0)
  



    uconv0 = Dropout(dropout_rate/2)(uconv0)
    #output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv0)    
    output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv0)    
    
    model = Model(input, output_layer)
    model.name = 'u-xception'

    return model

In [ ]:
def convolution_block(x, filters, size, strides=(1,1), padding='same', activation=True):
    x = Conv2D(filters, size, strides=strides, padding=padding)(x)
    x = BatchNormalization()(x)
    if activation == True:
        x = LeakyReLU(alpha=0.1)(x)
    return x

def residual_block(blockInput, num_filters=16):
    x = LeakyReLU(alpha=0.1)(blockInput)
    x = BatchNormalization()(x)
    blockInput = BatchNormalization()(blockInput)
    x = convolution_block(x, num_filters, (3,3) )
    x = convolution_block(x, num_filters, (3,3), activation=False)
    x = Add()([x, blockInput])
    return x

In [350]:
input_shape=(320,224,3)
encoder = EfficientNetB4(weights='imagenet',
                            include_top=False,
                            input_shape=input_shape)


Conv2D(start_neurons * 8, (3, 3))
inputs=keras.Input(shape=img_size + (3,))

# get indices in encoder net 
# convolve then deconvolve. 
# get residual blocks
l0=[l.name for l in encoder.layers]
l1 = [s.split(" ")[0] for s in l0]

# for layer_i in np.where(["c_add" in l for l in l1])[0]:
#     x=encoder.layers[layer_i].output
#     print(x.shape)
    

(None, 80, 56, 32)
(None, 40, 28, 56)
(None, 20, 14, 112)
(None, 20, 14, 160)
(None, 10, 7, 272)


In [465]:
np.where(["c_add" in l for l in l1])[0]

array([ 72, 131, 190, 278, 367], dtype=int64)

In [479]:
Conv2DTranspose(16,16)(keras.Input((224,224,3))).shape



# conv4 = backbone.layers[470].output
# conv4  
    
    
# conv4 = LeakyReLU(alpha=0.1)(conv4)
# pool4 = MaxPooling2D((2, 2))(conv4)
# pool4 = Dropout(dropout_rate)(pool4)
    
#     # Middle

        
        
# convm = Conv2D(start_neurons * 32, (3, 3), activation=None, padding="same")(pool4)



TensorShape([None, 239, 239, 16])

In [572]:
blocks_in_l1 = np.where(["block" in l for l in l1])[0]

l2 = [s.split("block")[1].split("_")[0][0] for s in [l1[b] for b in blocks_in_l1]]
    
l2
p=0
ll=[]
for i in range(len(l2)):
    
    if l2[i]!=p:
        p=l2[i]
        print(i)
        ll.append(i)

ll.append(len(l2))  


# [encoder.layers[l].name for l in ll ]


# [[l1[b] for b in blocks_in_l1][l] for l in ll]
# # # l2.count('1')


# [[l1[b] for b in blocks_in_l1][i-1] for i in ll[1:] ]
# #print(l2)
[encoder.layers[i+6].name for i in ll [1:]]

0
22
81
140
229
317
436


['block1b_add',
 'block2d_add',
 'block3d_add',
 'block4f_add',
 'block5f_add',
 'block6h_add',
 'block7b_add']

In [672]:
Conv2DTranspose(16,16)(keras.Input((224,224,3))).shape


conv0=encoder.layers[ll[-1]+6].output
print("conv0.shape: ",conv0.shape)
print("conv0: ",conv0)
for eff_block in reversed(ll[:]):
    print(eff_block)
    
    conv = encoder.layers[eff_block+6].output
    #concatenate()
    #print(conv.output.shape )
    #conv0 = Conv2DTranspose(12,(2,2),strides=(2,2))(conv0)#(conv.output)
    #print("a:",a.shape)
    print("conv: ",conv.shape)
    print("conv trans: ",Conv2DTranspose(12,(2,2),strides=(2,2))(conv).shape)
    #print("conv0: ",conv0.shape)
#     if eff_block in [317,229,81]:
#         a = Conv2DTranspose(12,(2,2),strides=(2,2))(conv0)#(conv.output)
#         print("*****")
#         print("a:",a.shape)
#         print("conv: ",conv.shape)
#         print("conv0: ",conv0.shape)
#         print("concat")
        
    #conv0=conv
  #  print("s1:",Conv2DTranspose(1,(2,2),strides=(2,2))(conv.output).shape)
    #print("s2:",Conv2DTranspose(1,(2,2),strides=(2,2))(conv0).shape)
    
        #conv0 = concatenate([a,conv])
    
    
    
    
    
#conv0.output.shape
[encoder.layers[eff_block+6].name for eff_block in  ll[1:]]
#ll





conv0.shape:  (None, 7, 7, 448)
conv0:  KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7, 448), dtype=tf.float32, name=None), name='block7b_add/add:0', description="created by layer 'block7b_add'")
464
conv:  (None, 7, 7, 448)
conv trans:  (None, 14, 14, 12)
436
conv:  (None, 7, 7, 272)
conv trans:  (None, 14, 14, 12)
317
conv:  (None, 14, 14, 160)
conv trans:  (None, 28, 28, 12)
229
conv:  (None, 14, 14, 112)
conv trans:  (None, 28, 28, 12)
140
conv:  (None, 28, 28, 56)
conv trans:  (None, 56, 56, 12)
81
conv:  (None, 56, 56, 32)
conv trans:  (None, 112, 112, 12)
22
conv:  (None, 112, 112, 24)
conv trans:  (None, 224, 224, 12)
0
conv:  (None, 112, 112, 48)
conv trans:  (None, 224, 224, 12)


['block1b_add',
 'block2d_add',
 'block3d_add',
 'block4f_add',
 'block5f_add',
 'block6h_add',
 'block7b_add']

In [686]:
#encoder7 to decoder7:
N=12
input_shape=(224,224,3)
encoder = EfficientNetB4(weights='imagenet',
                            include_top=False,
                            input_shape=input_shape)

input = encoder.input

enc7 = encoder.layers[ll[-1]+6].output
dec7 = Conv2DTranspose(N, (2,2),strides=(2,2))(enc7)

#concatenate encoder5:
enc5 = encoder.layers[ll[-3]+6].output

concat = concatenate([dec7,enc5])

dec5 = Conv2DTranspose(N, (2,2),strides=(2,2))(concat)
dec5 = Conv2D(N, (2,2),strides=(2,2))(dec5)
dec5 = Conv2DTranspose(N, (2,2),strides=(2,2))(dec5)

enc3 = encoder.layers[ll[-5]+6].output

concat = concatenate([dec5,enc3])

dec3 = Conv2DTranspose(N, (2,2),strides=(2,2))(concat)
dec3 = Conv2D(N, (2,2),strides=(2,2))(dec3)
dec3 = Conv2DTranspose(N, (2,2),strides=(2,2))(dec3)

dec3.shape


enc2 = encoder.layers[ll[-6]+6].output
concat = concatenate([dec3,enc2])

dec2 = Conv2DTranspose(N, (2,2),strides=(2,2))(concat)
dec2 = Conv2D(N, (2,2),strides=(2,2))(dec2)
dec2 = Conv2DTranspose(N, (2,2),strides=(2,2))(dec2)

enc0 = encoder.layers[ll[-7]+6].output
concat = concatenate([dec2,enc0])


dec0 = Conv2DTranspose(N, (2,2),strides=(2,2))(concat)
dec0 = Conv2D(N, (2,2),strides=(2,2))(dec0)
dec0 = Conv2DTranspose(N, (2,2),strides=(2,2))(dec0)

output = Conv2D(1, 3, padding="same", activation="sigmoid")(dec0)


input.shape
model = keras.Model(input, output)
model.summary()

Model: "model_19"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_190 (InputLayer)          [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling_18 (Rescaling)        (None, 224, 224, 3)  0           input_190[0][0]                  
__________________________________________________________________________________________________
normalization_18 (Normalization (None, 224, 224, 3)  7           rescaling_18[0][0]               
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization_18[0][0]           
___________________________________________________________________________________________

__________________________________________________________________________________________________
block6f_se_excite (Multiply)    (None, 7, 7, 1632)   0           block6f_activation[0][0]         
                                                                 block6f_se_expand[0][0]          
__________________________________________________________________________________________________
block6f_project_conv (Conv2D)   (None, 7, 7, 272)    443904      block6f_se_excite[0][0]          
__________________________________________________________________________________________________
block6f_project_bn (BatchNormal (None, 7, 7, 272)    1088        block6f_project_conv[0][0]       
__________________________________________________________________________________________________
block6f_drop (Dropout)          (None, 7, 7, 272)    0           block6f_project_bn[0][0]         
__________________________________________________________________________________________________
block6f_ad

In [685]:
encoder.layers[ll[-7]+6]



# model.weights[3]=np.mean(model.weights[3])
# model.weights

In [623]:
dir()

['AbstractRNNCell',
 'Activation',
 'ActivityRegularization',
 'Add',
 'AdditiveAttention',
 'AlphaDropout',
 'Attention',
 'Average',
 'AveragePooling1D',
 'AveragePooling2D',
 'AveragePooling3D',
 'AvgPool1D',
 'AvgPool2D',
 'AvgPool3D',
 'BatchNormalization',
 'Bidirectional',
 'BlockArgs',
 'BlockDecoder',
 'Concatenate',
 'Conv1D',
 'Conv1DTranspose',
 'Conv2D',
 'Conv2DTranspose',
 'Conv2DTranspose_block',
 'Conv3D',
 'Conv3DTranspose',
 'ConvLSTM2D',
 'Convolution1D',
 'Convolution1DTranspose',
 'Convolution2D',
 'Convolution2DTranspose',
 'Convolution3D',
 'Convolution3DTranspose',
 'Cropping1D',
 'Cropping2D',
 'Cropping3D',
 'Dense',
 'DenseFeatures',
 'DepthwiseConv2D',
 'Dot',
 'DoubleConv',
 'DropConnect',
 'Dropout',
 'ELU',
 'EfficientNetB4',
 'Embedding',
 'Flatten',
 'GRU',
 'GRUCell',
 'GaussianDropout',
 'GaussianNoise',
 'GlobalAveragePooling1D',
 'GlobalAveragePooling2D',
 'GlobalAveragePooling3D',
 'GlobalAvgPool1D',
 'GlobalAvgPool2D',
 'GlobalAvgPool3D',
 'Globa

In [ ]:
for i in [436,317,]

In [589]:
for eff_block in reversed(ll[:-2]):
    print(encoder.layers[eff_block+6].output.shape)

(None, 20, 14, 160)
(None, 20, 14, 112)
(None, 40, 28, 56)
(None, 80, 56, 32)
(None, 160, 112, 24)
(None, 160, 112, 48)


In [689]:
#encoder.summary()
log=dict()
log[0]=[]
log
np.random.randint(6)

4